In [21]:
from copy import deepcopy
import pygame
from game import Game
from myfile import *
import time
from GUI import *

pygame 2.3.0 (SDL 2.24.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [23]:
def simulate_move(piece, move, board, game, skip):
    """
    Simulates a move on the board by moving a piece, capturing any opponent's piece, and updating the game state.
    and return updated game board after the move.
    """
    board.move(piece, move[0], move[1])  # Move the piece on the board.
    if skip:
        board.remove(skip)  # Remove opponent's piece that was captured during the move.

    return board

In [24]:
def get_all_moves(board, color, game):
    """
    Gets all valid moves for a given color on the current game board.
    and returns a List of all possible game boards after making each move.
    """
    moves = []

    for piece in board.get_all_pieces(color):
        valid_moves = board.get_valid_moves(piece)  # Get all valid moves for the piece.
        for move, skip in valid_moves.items():
            temp_board = deepcopy(board)
            temp_piece = temp_board.get_piece(piece.row, piece.col)
            new_board = simulate_move(temp_piece, move, temp_board, game, skip)  # Simulate the move on the board.
            moves.append(new_board)
    
    return moves

In [25]:
def alpha_beta(position, depth, alpha, beta, max_player, game, human_play):
    """
    Implementation of the Alpha-Beta Pruning algorithm over minimax
    """
    if depth == 0:
        # If depth is 0, evaluate the current position and return the score along with the position.
        return position.evaluate(game.board.player), position

    if max_player:
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        best_move = None
        for move in get_all_moves(position, color_m, game):
            # Recursively call alpha_beta with the next move and update alpha value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, False, game, human_play)[0]
            if evaluation > alpha:
                alpha = evaluation
                best_move = move
            if alpha >= beta:
                # Perform pruning if alpha is greater than or equal to beta.
                break

        return alpha, best_move
    else:
        best_move = None
        for move in get_all_moves(position, game.turn, game):
            # Recursively call alpha_beta with the next move and update beta value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, True, game, human_play)[0]
            if evaluation < beta:
                beta = evaluation
                best_move = move
            if beta <= alpha:
                # Perform pruning if beta is less than or equal to alpha.
                break

        return beta, best_move

In [26]:
#INITIALLY CHECKING THE ALGORITHM USING THIS
def minimax(position, depth, max_player, game, human_play):
    """
    Implementation of the minimax algorithm for game AI.
    """
    if depth == 0:
        return position.evaluate(game.board.player), position  # Return evaluation of current position and the position itself
    
    if max_player:
        maxEval = float('-inf')  #
        best_move = None 
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        for move in get_all_moves(position, color_m, game): 
            evaluation = minimax(move, depth-1, False, game, human_play)[0] 
            maxEval = max(maxEval, evaluation)  # Update the maximum evaluation score
            if maxEval == evaluation:
                best_move = move  # Update the best move if a better move is found
        
        return maxEval, best_move  
    else:
        minEval = float('inf')  
        best_move = None  
        for move in get_all_moves(position, game.turn, game):  
            evaluation = minimax(move, depth-1, True, game, human_play)[0]  
            minEval = min(minEval, evaluation)  # Update the minimum evaluation score
            if minEval == evaluation:
                best_move = move  
        
        return minEval, best_move  # Return the minimum evaluation score and the best move found

In [27]:
#HUMAN VS AI
def human_AI(play1):
    player1 = play1
    player2 = None
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    
    while counter < 40:
        
        old_pieces = obj.board.red_left + obj.board.white_left
        if obj.turn == "red":
            value, new_board = alpha_beta(obj.get_board(), 2, float("-inf"), float("inf"), obj.turn, obj, True)
            #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, True)
            obj.ai_move(new_board)
            print(new_board.board)
            gui = Checkerboard(obj.board.board)

        else:
            gui = Checkerboard(obj.board.board)
            row, col = gui.run()
            obj.select(row, col)
            
            
        print(obj.turn)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "red":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [28]:
def AI_vs_AI(player_1, player_2):
    player1 = player_1
    player2 = player_2
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    while counter < 40:
        old_pieces = obj.board.red_left + obj.board.white_left
        # old_pieces = len(obj.board.get_all_pieces(obj.turn)) + len(obj.board.get_all_pieces(opponent))
        value, new_board = alpha_beta(obj.get_board(), 2, float("-inf"), float("inf"), obj.turn, obj, False)
        #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, False)
        #print(obj.turn)
        # print(new_board.board)
        obj.ai_move(new_board)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        # print(counter)
        # print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "red":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    #print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [29]:
def parentSelection(chromosomes, type):

    if type == "random":
        index = np.random.choice(chromosomes, 2, replace=False) 
        
        parent1 = index[0]
        parent2 = index[1]

        if parent1 == parent2:
            print("Parents are the same??")

    elif type == "truncation":
        sorted_population = sorted(chromosomes, key=lambda chromosome: chromosome.score, reverse=True)
        parent1 = sorted_population[-1]
        parent2 = sorted_population[-2]

    elif type == "fps":
        total_fitness = sum(chromosome.score for chromosome in chromosomes)
        chromosome_probabilities = [chromosome.fitness/total_fitness for chromosome in chromosomes]
        
        parent1 = random.choices(chromosomes, weights=chromosome_probabilities)
        parent2 = random.choices(chromosomes, weights=chromosome_probabilities)
        # # generate two random numbers between 0 and the total fitness
        # spin_1 = random.uniform(0, total_fitness)
        # spin_2 = random.uniform(0, total_fitness)

        # # select two chromosomes proportional to their fitness
        # partial_sum = 0
        # parent1 = None
        # parent2 = None
        # for chromosome in chromosomes:
        #     partial_sum += chromosome.score
        #     if partial_sum >= spin_1 and parent1 is None:
        #         parent1 = chromosome
        #     if partial_sum >= spin_2 and parent2 is None:
        #         parent2 = chromosome
        #     if parent1 is not None and parent2 is not None:
        #         break 

    elif type == "binary_tournament":
        parent_1 = random.choice(chromosomes)
        parent_2 = random.choice(chromosomes)

        # Choose the fittest chromosome as the first parent
        if parent_1.score > parent_2.score:
            parent1 = parent_1
        else:
            parent1 = parent_2

        # repeats for second parent
        parent_1 = random.choice(chromosomes)
        parent_2 = random.choice(chromosomes)
        if parent_1.score > parent_2.score:
            parent2 = parent_1
        else:
            parent2 = parent_2

    return parent1, parent2

In [30]:
def crossover(parent1, parent2):

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = parent1.getWeights()
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = parent2.getWeights()
    
    crossoverPoint1 = random.randint(0, len(p1first_layer_weights) - 1)

    o1first_layer_weights = []
    o2first_layer_weights = []

    for i in range(len(p1first_layer_weights)):
        
        o1 = np.concatenate((p1first_layer_weights[i][:crossoverPoint1],(p2first_layer_weights[i][crossoverPoint1:])))
        o2 = np.concatenate((p2first_layer_weights[i][:crossoverPoint1], p1first_layer_weights[i][crossoverPoint1:]))

        o1first_layer_weights.append(o1)
        o2first_layer_weights.append(o2)

    crossoverPoint2 = random.randint(0, len(p1first_layer_bias) - 1)
   
    o1first_layer_bias = []
    o2first_layer_bias = []

    for i in range(len(p1first_layer_bias)):

        o1 = np.concatenate((p1first_layer_bias[i][:crossoverPoint2], p2first_layer_bias[i][crossoverPoint2:]))
        o2 = np.concatenate((p2first_layer_bias[i][:crossoverPoint2], p1first_layer_bias[i][crossoverPoint2:]))

        o1first_layer_bias.append(o1)
        o2first_layer_bias.append(o2)

    crossoverPoint3 = random.randint(0, len(p1second_layer_weights) - 1)
   
    o1second_layer_weights = []
    o2second_layer_weights = []

    for i in range(len(p1second_layer_weights)):

        o1 = np.concatenate((p1second_layer_weights[i][:crossoverPoint3], p2second_layer_weights[i][crossoverPoint3:]))
        o2 = np.concatenate((p2second_layer_weights[i][:crossoverPoint3], p1second_layer_weights[i][crossoverPoint3:]))

        o1second_layer_weights.append(o1)
        o2second_layer_weights.append(o2)

    crossoverPoint4 = random.randint(0, len(p1second_layer_bias) - 1)

    o1second_layer_bias = []
    o2second_layer_bias = []

    for i in range(len(p1second_layer_bias)):

        o1 = np.concatenate((p1second_layer_bias[i][:crossoverPoint4], p2second_layer_bias[i][crossoverPoint4:]))
        o2 = np.concatenate((p2second_layer_bias[i][:crossoverPoint4], p1second_layer_bias[i][crossoverPoint4:]))

        o1second_layer_bias.append(o1)
        o2second_layer_bias.append(o2)

    crossoverPoint5 = random.randint(0, len(p1third_layer_weights) - 1)

    o1third_layer_weights = []
    o2third_layer_weights = []

    for i in range(len(p1third_layer_weights)):

        o1 = np.concatenate((p1third_layer_weights[i][:crossoverPoint5], p2third_layer_weights[i][crossoverPoint5:]))
        o2 = np.concatenate((p2third_layer_weights[i][:crossoverPoint5], p1third_layer_weights[i][crossoverPoint5:]))

        o1third_layer_weights.append(o1)
        o2third_layer_weights.append(o2)

    crossoverPoint6 = random.randint(0, len(p1third_layer_bias) - 1)

    o1third_layer_bias = []
    o2third_layer_bias = []

    for i in range(len(p1third_layer_bias)):

        o1 = np.concatenate((p1third_layer_bias[0][:crossoverPoint6], p2third_layer_bias[0][crossoverPoint6:]))
        o2 = np.concatenate((p2third_layer_bias[0][:crossoverPoint6], p1third_layer_bias[0][crossoverPoint6:]))

        o1third_layer_bias.append(o1)
        o2third_layer_bias.append(o2)

    offspring1 = []
    offspring1.append(o1first_layer_bias)
    offspring1.append(o1second_layer_bias)
    offspring1.append(o1third_layer_bias)

    offspring1.append(o1first_layer_weights)
    offspring1.append(o1second_layer_weights)
    offspring1.append(o1third_layer_weights)

    offspring2 = []
    offspring2.append(o2first_layer_bias)
    offspring2.append(o2second_layer_bias)
    offspring2.append(o2third_layer_bias)

    offspring2.append(o2first_layer_weights)
    offspring2.append(o2second_layer_weights)
    offspring2.append(o2third_layer_weights)

    return offspring1, offspring2

In [31]:
def mutationNew(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=10)


            for j in range(10):
                o2first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o2second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1third_layer_weights) - 1):
         if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o2third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [32]:
def mutation(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=2)
        o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=2)
        o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=2)
        o1second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        
        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=2)
        o2second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=2)
        o1third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=2)
        o2third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=2)
            o1first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=2)
            o2first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=2)
            o1second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=2)
            o2second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1third_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=2)
            o1third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=2)
            o2third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [33]:
def calculateFitness(population):

    for i in range(len(population) - 1):

        player1 = population[i]

        challengers = population[:i] +  population[i+1:]

        randomPlayers = np.random.choice(challengers, size=3, replace=False)

        for player2 in randomPlayers:
            obj = Game(player1, player2)
            new_board, winner, best_player = AI_vs_AI(player1, player2)

            if winner=="draw":
                player1.score +=1
                player2.score +=1
            else:
                best_player.score +=2

                if winner == 'red':
                    player2.score -= 1
                else:
                    player1.score -= 1

            #print("game complete")

In [34]:
def survivorSelection(newPopulation, type):

    finalPopulation = []
        
    if type == "truncation":

        tup = []
        for i in newPopulation:
            tup.append((i.score, i))
        
        sortedlist = sorted(tup, key=lambda x: x[0], reverse=True)

        for pop in sortedlist:
            finalPopulation.append(pop[1])

        finalPopulation =  finalPopulation[:10]
    
    elif type == "fps":
        total_fitness = sum(chromosome.score for chromosome in newPopulation)
        chromosome_probabilities = [chromosome.fitness/total_fitness for chromosome in newPopulation]
        for i in range(10):
            finalPopulation.append(np.random.choice(newPopulation, p=chromosome_probabilities))


    elif type == "binary_tournament":
         
        for i in range(10):
        # Choose two random parents from the population
            parent1 = random.choice(newPopulation)
            parent2 = random.choice(newPopulation)

            # Choose the fittest chromosome as the first parent
            if parent1.score > parent2.score:
                finalPopulation.append(parent1)
            else:
                finalPopulation.append(parent2)
    
    return finalPopulation

In [16]:
def NueroEvolution(population, generations):

    best_overall_player_score = 0

    chromosomes = []

    #Initialise random population
    for i in range(population):
        chromosomes.append(evolutionary_player(i))

    calculateFitness(chromosomes)

    for i in chromosomes:
        print(i.score)
        print(i.first_layer_weights)

    #print(chromosomes)
    count = 10
    
    for n in range(generations):
        print("Generation: ", n)

        parent1, parent2 = parentSelection(chromosomes, "random")

        print("Parent 1", parent1)
        print("Parent 2", parent2)

        offspring1, offspring2 = crossover(parent1, parent2)  
        print("Crossover complete")

        offspring1AfterMutation, offspring2AfterMutation = mutation(offspring1, offspring2, mutationRate=0.5)

        parent1player, parent2player = createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count)
        print("Mutation complete")

        newPopulation = chromosomes + [parent1player, parent2player]
        

        calculateFitness(newPopulation)
        print("Fitness Calculation complete")

        chromosomes = survivorSelection(newPopulation, "truncation")

        high = 0
        
        print("Generation Scores")
        for i in chromosomes:
            if i.score > high:
                high = i.score
                highchrome = i
            print(i.score)
        
        print("Best Score", high, "Name", highchrome)

        if highchrome == parent1 or highchrome == parent2:
            print("equal??")

        if high > best_overall_player_score:
            best_overall_player_score = high
            best_overall_player = highchrome

    


        count += 2

    return best_overall_player

In [39]:
#player = NueroEvolution(12, 1)

In [40]:
#new_board, winner, best_player = human_AI(player)
#new_board, winner, best_player = AI_vs_AI()
'''
for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")
print(best_player.third_layer_bias)'''

'\nfor i in new_board.board:\n    print(i)\nfor i in new_board.board:\n    for piece in i:\n        if piece !=0:\n            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)\nprint("WINNER COLOR:")\nprint(winner)\nprint("WINNER PLAYER NUM:")\nprint(best_player.number)\nprint("best first layer weights:")\nprint(best_player.first_layer_weights)\nprint("best first layer bias:")\nprint(best_player.first_layer_bias)\nprint("best second layer weights:")\nprint(best_player.second_layer_weights)\nprint("best second layer bias:")\nprint(best_player.second_layer_bias)\nprint("best third layer weights:")\nprint(best_player.third_layer_weights)\nprint("best third layer bias:")\nprint(best_player.third_layer_bias)'

In [35]:
def NueroEvolution2(population, generations):

    best_overall_player_score = 0

    chromosomes = []

    #Initialise random population
    for i in range(population):
        chromosomes.append(evolutionary_player(i))

    calculateFitness(chromosomes)

   # for i in chromosomes:
    #    print(i.score)
    #    print(i.first_layer_weights)

    #print(chromosomes)
    count = 10
    
    for n in range(generations):
        print("Generation: ", n)

        parent1, parent2 = parentSelection(chromosomes, "random")

        print("Parent 1", parent1)
        print("Parent 2", parent2)

        offspring1, offspring2 = crossover(parent1, parent2)  
        print("Crossover complete")

        offspring1AfterMutation, offspring2AfterMutation = mutationNew(offspring1, offspring2, mutationRate=0.5)

        parent1player, parent2player = createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count)
        print("Mutation complete")

        newPopulation = chromosomes + [parent1player, parent2player]
        

        calculateFitness(newPopulation)
        print("Fitness Calculation complete")

        chromosomes = survivorSelection(newPopulation, "truncation")

        high = 0
        
        print("Generation Scores")
        for i in chromosomes:
            if i.score > high:
                high = i.score
                highchrome = i
            print(i.score)
        
        print("Best Score", high, "Name", highchrome)

        if highchrome == parent1 or highchrome == parent2:
            print("equal??")

        if high > best_overall_player_score:
            best_overall_player_score = high
            best_overall_player = highchrome

    


        count += 2

    return best_overall_player

In [36]:
player2 = NueroEvolution2(12, 20)

Generation:  0
Parent 1 <myfile.Evol_Player object at 0x10cacfb10>
Parent 2 <myfile.Evol_Player object at 0x10cacf450>
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
21
20
16
14
9
6
4
3
2
0
Best Score 21 Name <myfile.Evol_Player object at 0x10cacc7d0>
Generation:  1
Parent 1 <myfile.Evol_Player object at 0x10ca6c750>
Parent 2 <myfile.Evol_Player object at 0x105e2df50>
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
30
24
22
20
8
7
7
5
4
4
Best Score 30 Name <myfile.Evol_Player object at 0x10cacc7d0>
Generation:  2
Parent 1 <myfile.Evol_Player object at 0x105e23d10>
Parent 2 <myfile.Evol_Player object at 0x10a3ec990>
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
31
27
23
21
18
11
10
10
7
7
Best Score 31 Name <myfile.Evol_Player object at 0x10cacc7d0>
Generation:  3
Parent 1 <myfile.Evol_Player object at 0x10a3ec990>
Parent 2 <myfile.Evol_Player object at 0x10cacfc50>
Crossover 

In [ ]:
import sys
Sample_array_2 = np.arange(100)
np.set_printoptions(threshold=sys.maxsize)
print(player2.first_layer_weights)

[[ 6.67156809e-01  1.69058782e-02 -2.05131788e+00  4.39186928e-01
   6.90392103e-01  9.94593608e-01 -1.22829191e+00 -1.61417058e-01
   1.38513066e+00 -4.07293482e-01  6.80148382e-01 -2.06311304e+00
   2.06051321e+00  1.33677899e+00 -8.39276347e-01  8.43655295e-01
   1.32231219e+00  3.67282698e-02 -2.54939323e-02  1.45443745e+00
   2.32543316e-01 -8.57542387e-01  3.47829601e-01  1.15000692e+00
  -2.15129681e-01 -7.80617280e-01  9.62749738e-01 -8.36293102e-01
   7.38418277e-01 -1.00529064e+00 -1.22016001e+00  9.40248544e-01
   3.82262425e-01 -4.73459846e-01  3.06157739e-01 -2.31837356e+00
  -4.00066870e-01 -4.89310309e-01  1.31863059e-02 -1.24805072e+00
   8.53442067e-01  1.32961895e+00 -4.45535616e-01  2.37319902e+00
   1.17087949e-01 -1.03402725e+00 -7.56662419e-01 -4.81983451e-02
   8.97260772e-01 -8.19475800e-01  9.15576958e-01  2.19804302e+00
   8.31326287e-01  1.36891715e-01 -1.24408091e+00  1.35341901e+00
  -6.24175282e-01  6.71826206e-01 -3.01913032e-01  1.64490832e-01
  -2.04164

In [ ]:
print(player2.second_layer_weights)

[[ 1.53766891e-01  8.49902063e-01  1.13710791e+00  1.14013714e+00
   4.21479152e-01 -7.07639451e-01  1.10261655e-01 -2.02350603e+00
   9.44862166e-01  7.30871439e-01 -2.29536050e+00  2.84390713e-01
   1.86747836e-01 -7.76694105e-01  9.86420110e-01  4.33779733e-01
  -8.62886613e-01  1.25147693e+00 -7.82042733e-01 -2.16375249e+00
  -3.04847233e-01 -8.09981428e-01 -1.15027292e-01 -1.89410724e+00
  -7.00833435e-01  1.91426394e+00  2.10599461e+00  9.33303905e-01
  -1.10954086e+00  5.88656203e-01  2.88585993e-02  1.34030940e+00
  -3.49047810e-01 -1.15747515e+00 -1.23750331e-01  5.61058793e-01
   1.35471412e-01  6.42033247e-02  1.81819029e+00 -8.86058752e-02]
 [ 1.18058013e-01  4.58294294e-02 -9.99589563e-01 -1.13008142e+00
  -8.25644003e-01 -8.58340584e-02  1.85423777e+00  1.55232146e-01
  -2.06188191e-01  1.45096810e+00  6.78712082e-02 -1.43920713e+00
  -1.93935683e+00 -5.26081524e-01  3.91686882e-02 -8.00304739e-01
   8.90383766e-01  1.53531633e+00 -1.32311451e-01 -7.24399968e-01
  -2.8451

In [ ]:
print(player2.third_layer_weights)

[[ 7.24325807e-01 -5.53327374e-01  9.42319979e-01  8.10803725e-01
  -7.86882210e-01 -8.60258310e-01  9.10693627e-02  2.28644237e-01
   6.37719984e-01 -6.34875918e-01]
 [-9.70982891e-01  8.32790559e-01  1.23179918e+00 -1.64748406e+00
  -3.94459429e-01  5.24939542e-01  1.34699282e+00  4.20160869e-01
   1.86082261e+00 -9.86077631e-01]
 [-8.84393498e-01 -1.00099794e+00  3.77015409e-01 -1.59708001e+00
   1.48682659e-01 -2.50853328e+00  2.51154912e-01 -4.70048739e-01
   8.95384070e-01  8.24404576e-01]
 [ 3.01517615e-01  1.64531938e+00 -3.72799028e+00  3.05660981e-01
  -5.46388891e-01  8.31282211e-02 -4.10882299e-01 -1.68448353e+00
   2.12784915e-01 -7.78520826e-01]
 [-8.34038946e-01 -5.23643525e-01 -4.61442565e-01 -8.95418023e-01
  -1.43802547e+00 -1.82480572e-03  2.28013139e-01 -1.11073985e+00
   2.37754225e+00  6.24527276e-01]
 [-1.71223004e-01  1.32144917e+00 -1.84715903e-01  1.62260473e-01
  -7.41966068e-01  1.23801623e+00  7.73812317e-02 -2.34993096e-01
  -7.59586683e-02 -1.36700380e+00

In [ ]:
print(player2.first_layer_bias)

[[ 1.1071615  -1.40476791 -1.26958743 -1.33134361  0.98883696  0.13835649
  -0.67430459 -1.19381456 -0.37019211  0.63684705  2.29538385 -0.91562008
  -0.90428779 -0.59032551 -2.38692172 -0.81938501 -0.42937686 -0.39173605
  -1.45179584  0.636647   -1.3419559   0.20459591 -0.64745482 -1.37229707
   0.2342692  -0.3348566   0.37088856  0.51379524  0.07094249  0.51081194
  -0.37737635  0.72871486 -1.16114533  0.53853875 -1.03235871  2.02123851
  -0.48141348  1.74926921 -1.10200536  1.29949644 -0.59557976 -0.44417938
  -0.21814012  0.26536149 -0.45226657 -0.92839186  0.19426597  0.88041037
  -1.05399141 -0.23108987 -0.06303607 -0.25420388 -0.30000136 -1.06373406
   0.21662823 -1.48773057  0.31153722 -0.15824374  0.99305855 -0.80085859
  -1.19164657 -0.05864504  0.04529242  0.56515436  0.13377561 -1.42225536
  -1.94287076 -1.2257146  -1.0739374   1.13399466  0.07873442 -0.43727287
  -1.36236881 -0.21333826  0.79411256 -0.5524699  -0.29512785  2.63995156
   1.37053894 -0.53241178  1.90331692 

In [ ]:
print(player2.second_layer_bias)

[[-0.26373574 -2.33172407  0.70672609 -0.11349002  1.73922951  0.92772011
   1.16975728 -0.23807347  0.95075234 -0.57943215 -0.13727541 -0.85765896
   0.35038262 -1.08704094 -0.01966317 -0.96530802  0.92267986  0.46264449
  -1.83322314 -0.17175386  0.08964014 -1.18154159  1.10886341 -1.51909956
   1.86877929  0.13293733 -0.08838328  0.63938411  0.85773898  1.96034021
  -0.09600175  0.97085775  0.50919213 -0.11650285  0.52218925 -0.51446567
   0.87621609 -0.42679053  0.12043251 -0.34335413]]


In [48]:
print(player2.third_layer_bias)

[[ 1.17602329  0.06412447  1.64418797  0.31487303 -1.05412539 -0.81070267
   1.09171646 -0.79225361 -1.0276857  -1.13257659]]


In [37]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")

you have selected the piece at (2,1)
white
DIFF:  0
you have selected the piece at (3,0)
red
DIFF:  0
[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, white, 0], [0, 0, 0, white, 0, white, 0, white], [white, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, red], [red, 0, red, 0, red, 0, 0, 0], [0, red, 0, red, 0, red, 0, red], [red, 0, red, 0, red, 0, red, 0]]
white
DIFF:  0
you have selected the piece at (2,5)
white
DIFF:  0
you have selected the piece at (3,6)
red
DIFF:  0
[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, white, 0], [0, 0, 0, white, 0, 0, 0, white], [white, 0, 0, 0, 0, 0, white, 0], [0, red, 0, 0, 0, 0, 0, red], [0, 0, red, 0, red, 0, 0, 0], [0, red, 0, red, 0, red, 0, red], [red, 0, red, 0, red, 0, red, 0]]
white
DIFF:  0
you have selected the piece at (1,2)
white
DIFF:  0
you have selected the piece at (2,1)
red
DIFF:  0
[[0, white, 0, white, 0, white, 0, white], [white, 0, 0, 0, white, 0, white, 0], [0, white, 0, white, 

TypeError: cannot unpack non-iterable NoneType object

: 

In [ ]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")